# COVID-19 Prediction

In [8]:
# import py packages
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import summary

from tqdm import tqdm

import math
import os
import csv


## Dataset 

In [10]:
class CovidDataset(Dataset):
    def __init__(self, x, y) -> None:
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, index):
        if self.y is None:
            return self.x[index]
        else:
            return self.x[index], self.y[index]
        
    def __len__(self):
        return len(self.x)   

## nn Model

In [11]:
class PredModel(nn.Module):
    def __init__(self,input_dim) -> None:
        super(PredModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
    
    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x